In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import polars as pl

In [4]:
import os
os.getcwd()

'/Users/ryant/Documents/Github/kaggle-challenges/optiver-realizedvolatilityprediction-2022'

# Train Dataset

In [24]:
train = pd.read_csv('data/train.csv')
train

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
...,...,...,...
428927,126,32751,0.003461
428928,126,32753,0.003113
428929,126,32758,0.004070
428930,126,32763,0.003357


In [28]:
stock_id = 0

book = pd.read_parquet(f'data/book_train.parquet/stock_id={stock_id}')
book.loc[:,'stock_id'] = 0
trade = pd.read_parquet(f'data/trade_train.parquet/stock_id={stock_id}')
trade.loc[:,'stock_id'] = 0
book.head(5)

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [27]:
trade.head(5)

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
